In [16]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
data_df = pd.read_excel('drive/MyDrive/FinAI /data/model_train_data_whole_1103.xlsx')

In [18]:
rating_list = ['AAApi','AAA', 'AAA-','AA+', 'AA','AA-','A+','A','A-','BBB+','BBB','BBB-','BB+','BB','BB-','B+','B','B-','CCC','CC','C']
macro_rating_list = [0,0,0,1,1,1,2,2,2,3,3,3,3,3,3,3,3,3,3,3,3]
rating_list_df = {'ratings': rating_list,'macro_ratings':macro_rating_list}
rating_list_df = pd.DataFrame(rating_list_df)
rating_dict = rating_list_df.reset_index().set_index(['ratings'])['macro_ratings'].to_dict()
data_df['sample ID'] = data_df.index
data_df['macro ratings number'] = data_df['信用评级'].apply(lambda x: rating_dict[x] )

In [19]:
data_df = data_df.drop(columns = ['Unnamed: 0','full name','short name','wind code','最新报告期','信用评级发布日期', '信用评级', '信用评级与报告差距天数','freeCashFlowOperatingCashFlowRatio','日期差距最小值','ratings number'])

In [20]:
data_df

,currentRatio,quickRatio,cashRatio,daysOfSalesOutstanding,netProfitMargin,pretaxProfitMargin,grossProfitMargin,operatingProfitMargin,returnOnAssets,returnOnEquity,...,freeCashFlowPerShare,cashPerShare,companyEquityMultiplier,ebitPerRevenue,enterpriseValueMultiple,operatingCashFlowPerShare,operatingCashFlowSalesRatio,payablesTurnover,sample ID,macro ratings number
0,1.0040,0.8277,0.420467,40.7535,5.7830,8.0277,12.2566,4.5890,0.7504,3.1258,...,-1.8626,-0.2952,2.6047,0.011695,2.6047,0.2754,0.0435,1.526650,0,0
1,2.2105,1.7346,0.562841,85.2596,9.8876,9.7200,23.3548,9.7230,1.3206,1.8406,...,-0.8343,-0.5343,1.4668,0.028988,1.4668,0.0616,0.0333,2.667687,1,1
2,1.7518,1.2222,0.452765,21.3164,1.2564,1.6673,5.6689,1.4954,0.8661,1.8318,...,-1.1494,-0.7270,2.1284,0.004805,2.1284,-0.8031,-0.2003,13.848021,2,1
3,1.8327,1.7972,1.312670,8.5061,4.0169,6.0547,7.9779,5.4379,1.2391,2.5866,...,0.5398,-0.5288,1.9350,0.017147,1.9350,0.0292,0.0109,6.991825,3,1
4,1.2383,0.7997,0.135562,154.7455,4.0207,4.6444,8.3999,3.5450,1.4226,2.9554,...,-0.4452,-0.0693,2.0704,0.007667,2.0704,0.3589,0.0447,1.233116,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1912,1.4949,0.9001,0.263400,54.8045,14.6469,15.3034,27.8289,14.8035,14.0240,0.0000,...,-1.0523,-0.0724,2.3254,0.174172,0.0000,0.1984,0.0378,3.978500,1912,1
1913,2.0801,1.8319,1.331000,92.1187,14.9464,20.5480,26.9164,17.3061,6.8432,0.0000,...,-1.7166,2.1278,1.6026,0.176690,1.6026,-0.1500,-0.0717,6.088400,1913,2
1914,1.2618,1.1898,0.635200,20.6773,16.3158,23.0460,29.2711,22.1676,13.2859,0.0000,...,0.5934,0.2661,1.6863,0.190668,1.6863,1.3600,0.5808,3.643100,1914,0
1915,0.3322,0.2990,0.141000,38.9706,14.3447,24.9418,26.9058,18.9167,3.4248,10.3661,...,-0.7843,-0.0040,3.3726,0.197980,3.3726,0.8700,0.3203,2.493600,1915,1


In [21]:
data_df0 = data_df[data_df['macro ratings number']==0]
data_df1 = data_df[data_df['macro ratings number']==1]
data_df2 = data_df[data_df['macro ratings number']==2]
data_df3 = data_df[data_df['macro ratings number']==3]
data_df0_sample = data_df0.sample(200)
data_df1_sample = data_df1.sample(300)
data_df2_sample = data_df2.sample(200)
data_df3_sample = data_df3
data_df_sample = pd.concat([data_df0_sample,data_df1_sample,data_df2_sample,data_df3_sample])
data_df_sample.sample(frac = 1)

test_set_df = data_df_sample.sample(80)
test_set_df['is test data']=True
test_set_df = test_set_df[['is test data','sample ID']]
data_df = pd.merge(data_df,test_set_df,on = ['sample ID'],how = 'left')
data_df[data_df['is test data']==True]

train_data_df = data_df[data_df['is test data']!=True]
test_data_df = data_df[data_df['is test data']==True]
train_data_df = train_data_df.drop(columns = ['sample ID','is test data'])
test_data_df = test_data_df.drop(columns = ['sample ID','is test data'])

In [40]:
test_labels = test_data_df['macro ratings number']

In [22]:
data_df0 = train_data_df[train_data_df['macro ratings number']==0]
data_df1 = train_data_df[train_data_df['macro ratings number']==1]
data_df2 = train_data_df[train_data_df['macro ratings number']==2]
data_df3 = train_data_df[train_data_df['macro ratings number']==3]
col_dict = {}


def prepare_training_samples_df(data_df0,data_df1,data_df2,data_df3,sample_idx,col_dict):

    data_df0_sample = data_df0.sample(200)
    data_df1_sample = data_df1.sample(300)
    data_df2_sample = data_df2.sample(200)
    data_df3_sample = data_df3
    data_df_sample = pd.concat([data_df0_sample,data_df1_sample,data_df2_sample,data_df3_sample])
    data_df_sample.sample(frac = 1)
    labels = data_df_sample[['macro ratings number']].values
    data_df_sample = data_df_sample.drop(columns = ['macro ratings number'])
    data_df_sample = data_df_sample.sample(n=5,axis='columns')
    data_df_sample['labels'] = labels
    

    col_names = data_df_sample.columns.values.tolist()
    col_dict[str(sample_idx)] = col_names[:-1]
    columns = {}
    for idx, n in enumerate(col_names):
        if idx == len(col_names)-1:
            columns[n] = "y"
        else:
            columns[n] = "x%d"%idx
    data_df_sample = data_df_sample.rename(columns=columns)
    x_train = data_df_sample[[i for i in data_df_sample.columns if i!='y']]
    y_train = data_df_sample['y']
    return x_train, y_train, col_dict


def prepare_testing_samples_df(test_df):
    
    col_names = test_df.columns.values.tolist()
    columns = {}
    for idx, n in enumerate(col_names):
        if idx == len(col_names)-1:
            columns[n] = "y"
        else:
            columns[n] = "x%d"%idx
    test_data_sample_df = test_df.rename(columns=columns)
    x_test = test_data_sample_df[[i for i in test_data_sample_df.columns if i!='y']]
    y_test = test_data_sample_df['y']
    return x_test, y_test



In [23]:
x_train1, y_train1 ,col_dict= prepare_training_samples_df(data_df0,data_df1,data_df2,data_df3,1,col_dict)
x_train2, y_train2 ,col_dict= prepare_training_samples_df(data_df0,data_df1,data_df2,data_df3,2,col_dict)
x_train3, y_train3 ,col_dict= prepare_training_samples_df(data_df0,data_df1,data_df2,data_df3,3,col_dict)
x_train4, y_train4 ,col_dict= prepare_training_samples_df(data_df0,data_df1,data_df2,data_df3,4,col_dict)
x_train5, y_train5 ,col_dict= prepare_training_samples_df(data_df0,data_df1,data_df2,data_df3,5,col_dict)
x_train6, y_train6 ,col_dict= prepare_training_samples_df(data_df0,data_df1,data_df2,data_df3,6,col_dict)
x_train7, y_train7 ,col_dict= prepare_training_samples_df(data_df0,data_df1,data_df2,data_df3,7,col_dict)
x_train8, y_train8 ,col_dict= prepare_training_samples_df(data_df0,data_df1,data_df2,data_df3,8,col_dict)
x_train9, y_train9 ,col_dict= prepare_training_samples_df(data_df0,data_df1,data_df2,data_df3,9,col_dict)
x_train10, y_train10 ,col_dict= prepare_training_samples_df(data_df0,data_df1,data_df2,data_df3,10,col_dict)

In [36]:
col_dict['1']

['pretaxProfitMargin',
 'currentRatio',
 'enterpriseValueMultiple',
 'returnOnEquity',
 'companyEquityMultiplier']

In [25]:
x_test, y_test = prepare_testing_samples_df(test_data_df)

In [30]:
neurons1 = 128
neurons2 = 128
neurons3 = 64
neurons4 = 64
neurons5 = 32
neurons6 = 16

model1 = tf.keras.models.Sequential([
tf.keras.layers.Dense(neurons1,activation = 'relu'),
tf.keras.layers.Dense(neurons2,activation = 'relu'),
tf.keras.layers.Dense(neurons3,activation = 'relu'),
tf.keras.layers.Dense(neurons4,activation = 'relu'),
tf.keras.layers.Dense(neurons5,activation = 'relu'),
tf.keras.layers.Dense(neurons6,activation = 'relu'),
tf.keras.layers.Dense(4,activation = 'softmax')
])

model2 = tf.keras.models.Sequential([
tf.keras.layers.Dense(neurons1,activation = 'relu'),
tf.keras.layers.Dense(neurons2,activation = 'relu'),
tf.keras.layers.Dense(neurons3,activation = 'relu'),
tf.keras.layers.Dense(neurons4,activation = 'relu'),
tf.keras.layers.Dense(neurons5,activation = 'relu'),
tf.keras.layers.Dense(neurons6,activation = 'relu'),
tf.keras.layers.Dense(4,activation = 'softmax')
])

model3 = tf.keras.models.Sequential([
tf.keras.layers.Dense(neurons1,activation = 'relu'),
tf.keras.layers.Dense(neurons2,activation = 'relu'),
tf.keras.layers.Dense(neurons3,activation = 'relu'),
tf.keras.layers.Dense(neurons4,activation = 'relu'),
tf.keras.layers.Dense(neurons5,activation = 'relu'),
tf.keras.layers.Dense(neurons6,activation = 'relu'),
tf.keras.layers.Dense(4,activation = 'softmax')
])

model4 = tf.keras.models.Sequential([
tf.keras.layers.Dense(neurons1,activation = 'relu'),
tf.keras.layers.Dense(neurons2,activation = 'relu'),
tf.keras.layers.Dense(neurons3,activation = 'relu'),
tf.keras.layers.Dense(neurons4,activation = 'relu'),
tf.keras.layers.Dense(neurons5,activation = 'relu'),
tf.keras.layers.Dense(neurons6,activation = 'relu'),
tf.keras.layers.Dense(4,activation = 'softmax')
])

model5 = tf.keras.models.Sequential([
tf.keras.layers.Dense(neurons1,activation = 'relu'),
tf.keras.layers.Dense(neurons2,activation = 'relu'),
tf.keras.layers.Dense(neurons3,activation = 'relu'),
tf.keras.layers.Dense(neurons4,activation = 'relu'),
tf.keras.layers.Dense(neurons5,activation = 'relu'),
tf.keras.layers.Dense(neurons6,activation = 'relu'),
tf.keras.layers.Dense(4,activation = 'softmax')
])

model6 = tf.keras.models.Sequential([
tf.keras.layers.Dense(neurons1,activation = 'relu'),
tf.keras.layers.Dense(neurons2,activation = 'relu'),
tf.keras.layers.Dense(neurons3,activation = 'relu'),
tf.keras.layers.Dense(neurons4,activation = 'relu'),
tf.keras.layers.Dense(neurons5,activation = 'relu'),
tf.keras.layers.Dense(neurons6,activation = 'relu'),
tf.keras.layers.Dense(4,activation = 'softmax')
])

model7 = tf.keras.models.Sequential([
tf.keras.layers.Dense(neurons1,activation = 'relu'),
tf.keras.layers.Dense(neurons2,activation = 'relu'),
tf.keras.layers.Dense(neurons3,activation = 'relu'),
tf.keras.layers.Dense(neurons4,activation = 'relu'),
tf.keras.layers.Dense(neurons5,activation = 'relu'),
tf.keras.layers.Dense(neurons6,activation = 'relu'),
tf.keras.layers.Dense(4,activation = 'softmax')
])

model8 = tf.keras.models.Sequential([
tf.keras.layers.Dense(neurons1,activation = 'relu'),
tf.keras.layers.Dense(neurons2,activation = 'relu'),
tf.keras.layers.Dense(neurons3,activation = 'relu'),
tf.keras.layers.Dense(neurons4,activation = 'relu'),
tf.keras.layers.Dense(neurons5,activation = 'relu'),
tf.keras.layers.Dense(neurons6,activation = 'relu'),
tf.keras.layers.Dense(4,activation = 'softmax')
])

model9 = tf.keras.models.Sequential([
tf.keras.layers.Dense(neurons1,activation = 'relu'),
tf.keras.layers.Dense(neurons2,activation = 'relu'),
tf.keras.layers.Dense(neurons3,activation = 'relu'),
tf.keras.layers.Dense(neurons4,activation = 'relu'),
tf.keras.layers.Dense(neurons5,activation = 'relu'),
tf.keras.layers.Dense(neurons6,activation = 'relu'),
tf.keras.layers.Dense(4,activation = 'softmax')
])

model10 = tf.keras.models.Sequential([
tf.keras.layers.Dense(neurons1,activation = 'relu'),
tf.keras.layers.Dense(neurons2,activation = 'relu'),
tf.keras.layers.Dense(neurons3,activation = 'relu'),
tf.keras.layers.Dense(neurons4,activation = 'relu'),
tf.keras.layers.Dense(neurons5,activation = 'relu'),
tf.keras.layers.Dense(neurons6,activation = 'relu'),
tf.keras.layers.Dense(4,activation = 'softmax')
])


In [42]:
model_list = [model1,model2,model3,model4,model5,model6,model7,model8,model9,model10]

In [51]:
df_random_forest_results = pd.DataFrame()

In [52]:
for i in range(1,11):
    test_curr = test_data_df[col_dict[str(i)]]
    
    test_curr_mat = test_curr.values
    # print(np.argmax(model_list[i-1].predict(test_curr_mat),axis = 1))
    df_random_forest_results['tree' + str(i)+'results'] = np.argmax(model_list[i-1].predict(test_curr_mat),axis = 1)
    # print(model_list[i-1].predict(test_curr_mat))
    # print(test_curr_mat)

3/3 [==============================] - 0s 9ms/step


In [27]:

class myCallback(tf.keras.callbacks.Callback): 
  def on_epoch_end(self, epoch, logs={}): 
    if(logs.get('accsparse_categorical_accuracyuracy') > 0.9):
    # print("\nReached %2.2f%% accuracy, so stopping training!!" %(ACCURACY_THRESHOLD*100))   
      self.model.stop_training = True
callbacks = myCallback()

In [31]:
# callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10)
model1.compile(
    optimizer=keras.optimizers.Adam(lr=1e-3),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

model2.compile(
    optimizer=keras.optimizers.Adam(lr=1e-3),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

model3.compile(
    optimizer=keras.optimizers.Adam(lr=1e-3),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)



model4.compile(
    optimizer=keras.optimizers.Adam(lr=1e-3),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)


model5.compile(
    optimizer=keras.optimizers.Adam(lr=1e-3),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

model6.compile(
    optimizer=keras.optimizers.Adam(lr=1e-3),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)


model7.compile(
    optimizer=keras.optimizers.Adam(lr=1e-3),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

model8.compile(
    optimizer=keras.optimizers.Adam(lr=1e-3),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)


model9.compile(
    optimizer=keras.optimizers.Adam(lr=1e-3),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)


model10.compile(
    optimizer=keras.optimizers.Adam(lr=1e-3),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)






/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [32]:

max_epoch_size = 400
history1 = model1.fit(
    x_train1,
    y_train1,
    epochs=max_epoch_size
    # batch_size = 1

    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    # callbacks=[callbacks]
    # validation_data=(x_test, y_test),
)

history2 = model2.fit(
    x_train2,
    y_train2,
    epochs=max_epoch_size

)

history3 = model3.fit(
    x_train3,
    y_train3,
    epochs=max_epoch_size

)

history4 = model4.fit(
    x_train4,
    y_train4,
    epochs=max_epoch_size

)

history5 = model5.fit(
    x_train5,
    y_train5,
    epochs=max_epoch_size

)

history6 = model6.fit(
    x_train6,
    y_train6,
    epochs=max_epoch_size

)

history7 = model7.fit(
    x_train7,
    y_train7,
    epochs=max_epoch_size

)

history8 = model8.fit(
    x_train8,
    y_train8,
    epochs=max_epoch_size

)

history9 = model9.fit(
    x_train9,
    y_train9,
    epochs=max_epoch_size

)

history10 = model10.fit(
    x_train10,
    y_train10,
    epochs=max_epoch_size

)




流式输出内容被截断，只能显示最后 5000 行内容。
25/25 [==============================] - 0s 6ms/step - loss: 1.0384 - sparse_categorical_accuracy: 0.5020
Epoch 302/400
25/25 [==============================] - 0s 6ms/step - loss: 0.9667 - sparse_categorical_accuracy: 0.5436
Epoch 303/400
25/25 [==============================] - 0s 6ms/step - loss: 0.9103 - sparse_categorical_accuracy: 0.5553
Epoch 304/400
25/25 [==============================] - 0s 6ms/step - loss: 0.9585 - sparse_categorical_accuracy: 0.5722
Epoch 305/400
25/25 [==============================] - 0s 6ms/step - loss: 0.9109 - sparse_categorical_accuracy: 0.5800
Epoch 306/400
25/25 [==============================] - 0s 6ms/step - loss: 0.8831 - sparse_categorical_accuracy: 0.5761
Epoch 307/400
25/25 [==============================] - 0s 6ms/step - loss: 0.8800 - sparse_categorical_accuracy: 0.5839
Epoch 308/400
25/25 [==============================] - 0s 6ms/step - loss: 0.8846 - sparse_categorical_accuracy: 0.5891
Epoch 309/400
25/25 [======

In [34]:
x_test

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23
23,0.4826,0.4327,0.187836,48.9130,4.1638,9.2557,8.0964,4.4445,0.3954,1.3481,...,67.6751,15.0,-0.4598,0.0423,3.0936,0.000508,3.0936,0.195800,0.0564,1.276431
29,0.6555,0.6476,0.506739,23.0882,-20.3079,-13.1671,5.7628,-22.7660,-0.9233,-2.1153,...,57.9180,25.0,-1.9990,-0.5150,2.3763,-0.046558,2.3763,0.094400,0.0206,11.430601
30,1.8318,1.1175,1.041619,0.7276,0.4316,4.7898,28.1189,3.2781,0.1476,0.5138,...,58.3315,0.0,-0.7949,-0.6977,2.3999,0.024059,2.3999,-0.689500,-0.0686,3.593624
38,0.1558,0.1320,0.005661,490.1961,-287.6041,-132.2968,-139.5211,-345.3752,-1.5608,-20.5880,...,93.1900,0.0,0.0726,-0.0137,14.6843,0.000000,14.6843,-0.010100,-0.0010,0.097698
45,1.2430,0.4712,0.141784,24.6827,7.8183,13.1537,0.0000,11.6971,1.3520,14.3362,...,84.5703,0.0,-21.2735,-0.8726,6.4810,0.119416,6.4810,0.468931,0.0079,1.275841
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1802,0.8055,0.6104,0.262200,43.9067,5.2566,9.8497,14.0148,6.8942,0.9388,3.0960,...,70.3176,15.0,-0.7038,0.2100,3.3690,0.021014,3.3690,-1.330100,-0.1265,0.709300
1817,2.5124,2.2533,1.432300,194.0910,24.5628,37.5970,61.9996,30.7243,2.4485,4.0144,...,32.6344,15.0,-0.5501,0.8109,1.4844,0.214293,1.4844,0.014800,0.0116,1.448000
1819,1.2300,1.0112,0.252300,201.0050,5.0943,7.6104,21.5470,5.9036,1.5605,3.5556,...,52.7437,25.0,-0.0606,0.0099,2.1161,0.037995,2.1161,0.001400,0.0009,1.458900
1839,0.8391,0.8364,0.478800,11.2922,3.3572,9.9064,38.6105,5.4312,0.5028,0.3153,...,46.4405,25.0,0.2594,0.1966,1.8671,0.026619,1.8671,0.400000,0.1572,2.469100


In [ ]:
np.argmax(model.predict(x_test),axis = 1)

In [ ]:
y_test

In [ ]:
np.argmax(model.predict(x_train),axis = 1)

22/22 [==============================] - 0s 3ms/step


array([3, 0, 1, 3, 0, 0, 2, 1, 1, 3, 2, 2, 0, 1, 2, 2, 1, 0, 0, 0, 1, 1,
       1, 2, 1, 0, 1, 0, 1, 0, 2, 1, 3, 1, 1, 1, 0, 2, 1, 1, 1, 1, 2, 3,
       1, 3, 3, 2, 2, 3, 1, 2, 2, 2, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1,
       3, 2, 1, 0, 2, 0, 0, 2, 1, 0, 0, 2, 1, 1, 0, 0, 2, 2, 2, 1, 0, 2,
       0, 0, 1, 2, 0, 1, 1, 0, 0, 0, 3, 2, 1, 1, 1, 1, 1, 2, 1, 2, 2, 1,
       0, 1, 2, 1, 1, 0, 2, 2, 3, 0, 2, 0, 0, 2, 0, 1, 1, 1, 3, 0, 0, 0,
       0, 2, 0, 1, 2, 2, 0, 0, 2, 0, 2, 2, 1, 1, 0, 2, 1, 1, 0, 3, 1, 2,
       1, 1, 1, 3, 0, 0, 1, 0, 2, 1, 2, 1, 0, 0, 1, 1, 1, 3, 2, 2, 1, 3,
       0, 2, 2, 0, 3, 1, 3, 1, 1, 0, 1, 1, 3, 2, 1, 1, 3, 2, 3, 1, 3, 1,
       2, 1, 1, 0, 0, 1, 3, 2, 1, 3, 0, 1, 0, 1, 3, 1, 1, 1, 2, 1, 2, 2,
       2, 3, 0, 1, 0, 0, 1, 0, 2, 1, 0, 2, 3, 2, 2, 1, 3, 2, 2, 3, 2, 0,
       2, 1, 1, 2, 0, 2, 1, 1, 1, 0, 1, 3, 2, 1, 1, 0, 2, 1, 1, 3, 1, 1,
       0, 0, 0, 0, 3, 1, 2, 0, 3, 1, 1, 1, 3, 0, 2, 0, 1, 1, 0, 0, 1, 1,
       2, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 2, 2, 0, 0,

In [ ]:
y_train - np.argmax(model.predict(x_train),axis = 1)

22/22 [==============================] - 0s 2ms/step


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [ ]:
y_test - np.argmax(model.predict(x_test),axis = 1)

3/3 [==============================] - 0s 5ms/step


array([ 0.,  0.,  0., -1.,  1., -1.,  0.,  0.,  0.,  0.,  0.,  1., -1.,
        0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  1.,  0.,  1.,
        0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
        0.,  0.,  0., -2.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  1.,
        0., -1., -1.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,
        0.,  2.,  1., -1.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
        1.])

In [ ]:
data_df.sample(n=5,axis='columns')